MODS Phenotypes: Step 5. Grady Sub-Graph Mining
===

# `imports`

In [1]:
import os
import sys
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import multiprocessing
import dill as pickle
from tqdm.auto import tqdm
from multiprocessing import Pool

n_cpus = 180

***TODO***: Instead of inserting the sys path to the local code, pip install the repo to a `mods` environment

In [2]:
sys.path.insert(0, "/opt/scratchspace/KLAB_SAIL/MODSPhenotypes/gSpan/")
import shlex

In [3]:
from src.gspan_mining.main import parse_args, main
from src.gspan_mining.data_processing import process_graph_data

In [4]:
site_name = 'grady'
run_id = '2023_07_29'

# 1. Process the saved `pd.DataFrame()` on disk to our `.gSpan` format.

In [5]:
data_fp = Path(f"/opt/scratchspace/KLAB_SAIL/MODSPhenotypes/data/{run_id}/{site_name}_{run_id}_sofa_scores_df.parquet")

In [6]:
output_dir = Path.home() / 'tmp' / 'gspan_output'
output_dir.mkdir(exist_ok=True, parents=True)

In [7]:
%%time
# TODO: benchmark the wall clock time on this call
file_paths = process_graph_data(data_fp, output_dir, n_cpus)

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/19591 [00:00<?, ?it/s]

  0%|          | 0/19591 [00:00<?, ?it/s]

  0%|          | 0/19591 [00:00<?, ?it/s]

  0%|          | 0/19591 [00:00<?, ?it/s]

  0%|          | 0/19591 [00:00<?, ?it/s]

  0%|          | 0/19591 [00:00<?, ?it/s]

CPU times: user 29.8 s, sys: 37.9 s, total: 1min 7s
Wall time: 1min 17s


# 2. Frequent Subgraph Mining (FSM) with gSpan

### Parse Data to Graph

In [8]:
# We define our support using the ratio of 20 for 20k patients from literature
support = int((20/20_000)*55799)
min_subgraph_len = 3
max_subgraph_len = 6

In [9]:
arg_string = f"-fp '{file_paths[0].parent}' -s {support} -l {min_subgraph_len} -u {max_subgraph_len}"
args = parse_args(shlex.split(arg_string))

In [10]:
args

Namespace(gspan_fp=PosixPath('/home/gmatlin/tmp/gspan_output'), support=55, lower=3, upper=6)

In [11]:
%%time
# TODO: SPEED UP REPORTING IN GRAPH MINING
# TODO: ADD PRINT STATEMENTS TO INDICATE HOW FAR ALONG
# TODO: TQDM FOR THE REPORT LOOPS
# TODO: PANDAS ALTERNATIVE FOR THE REPORTING SLOG??

graph_miner = main(args) # WARNING: TAKES A FEW HOURS (?)

Running `gSpan` package on each `.gSpan` file:: 0it [00:00, ?it/s]

Read:	3.52 s
Mine:	20.69 s
Total:	24.21 s
Read:	3.03 s
Mine:	19.419999999999998 s
Total:	22.45 s
Read:	3.59 s
Mine:	19.29 s
Total:	22.88 s
Read:	2.95 s
Mine:	20.27 s
Total:	23.22 s
Read:	3.22 s
Mine:	17.89 s
Total:	21.11 s
Read:	3.47 s
Mine:	18.580000000000002 s
Total:	22.05 s


Building a report from each column result:   0%|          | 0/6 [00:00<?, ?it/s]

CONCAT DATA
CPU times: user 7min 16s, sys: 1.74 s, total: 7min 18s
Wall time: 7min 19s


##### Save `graph_miner`

In [12]:
with open(f"/home/gmatlin/tmp/{site_name}_{run_id}_graph_miner.pkl", "wb") as dill_file:
    pickle.dump(graph_miner, dill_file)

##### Load `graph_miner`